# Preparation

In [2]:
## If using Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# install dependencies

!pip install pyshacl
!pip install rdflib
!pip install sparqlwrapper

import regex as re
import time

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pyshacl import validate
from rdflib import Graph, URIRef, BNode, Literal 
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data Collection

In [2]:
def query_sparql(query, sparql_endpoint):
  """
  Query to certain SPARQL endpoint, such as Wikidata SPARQL.

  Parameters
  ----------
  query : str
    A SPARQL query to be run
  sparql_endpoint : str
    A SPARQL API endpoint 

  Returns
  -------
  DataFrame
    A table consisting of instances to be validated
  """

  # set up the query
  sparqlwd = SPARQLWrapper(sparql_endpoint)
  sparqlwd.setQuery(query)
  sparqlwd.setReturnFormat(JSON)

  # get the data and transform the result into pandas dataframe
  while True:
    try:
      results = sparqlwd.query().convert()
      results_df = pd.json_normalize(results['results']['bindings'])
      break
    except:
      continue
  
  # return the result in dataframe
  return results_df

First of all, get all the entities which will be validated using SHACL.

In [ ]:
# replace the query as you defined below
query = """
...
"""

# example: retrieve all instances of Country class with a limit of 1000
query = """
SELECT DISTINCT ?entity
WHERE {
    ?entity a dbo:Country .
}
LIMIT 1000
"""

# execute the query using query_sparql function
data = query_sparql(query, "http://dbpedia.org/sparql")
data.to_csv("DEFINED/PATH", index=False)
data.head()

Then, get all the required property values for all entities.

In [ ]:
def get_data_prop(df, prop_list, window_size, sparql_endpoint):
  """
  Query the property value given all the instances to be validated.

  Parameters
  ----------
  df : DataFrame
    A table containing all the instances to be validated
  prop_list : list
    A list of properties to be checked
  window_size : int
    A number of data instances used in one query
  sparql_endpoint : str
    A SPARQL API endpoint 

  Returns
  -------
  DataFrame
    A table consisting of all the properties of instances along with their values
  """

  # initiate the variables
  size = df.shape[0]
  list_data = []

  for prop in prop_list:
    # initiate index
    idx_lower = 0
    idx_upper = window_size

    # window-looping through length of data
    while idx_lower <= size:
      if idx_upper > size:
        idx_upper = size

      # check the index
      print(idx_lower, idx_upper)

      while True:
        try:
          query = f"""
SELECT ?s ?p ?o
WHERE {{
  VALUES ?s {{{' '.join(data['entity'][idx_lower:idx_upper]) }}}
  BIND({prop} AS ?p)
  ?s ?p ?o .
}}
"""
          res = query_sparql(query, sparql_endpoint)
          list_data.append(res)

        except:
          time.sleep(5)
          continue

        # update idx
        idx_lower += window_size
        idx_upper += window_size

        break

  return pd.concat(list_data, ignore_index=True, sort=False)

In [ ]:
# initiate data and variable before function call
data['entity'] = data['entity.value'].apply(lambda x: f"<{x}>")
# SPARQL endpoint URL i.e. http://dbpedia.org/sparql
endpoint = "..."
# list all the property to be visualized
prop_list = ['...']

# execute the query to get all the property value
data_prop = get_data_prop(data, prop_list, 50, endpoint)

# Shapes Generation

## Manual

In [ ]:
shapes_graph = \
"""
...
"""

# example: the shapes graph to check all instances of Country class should
# have a label and description property
shapes_graph = \
"""
@prefix : <http://example.org/ns#> .
@prefix dash: <http://datashapes.org/dash#> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbp: <http://dbpedia.org/property/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/ns#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .

ex:CountryLabelDescriptionShape
    a sh:NodeShape;
    sh:targetClass dbo:Country ;
    sh:property [
        sh:path rdfs:label ;
        sh:minCount 1 ;
    ] ;
    sh:property [
        sh:path rdfs:comment ;
        sh:minCount 1 ;
    ] ;
    sh:property [
        sh:path dbo:abstract ;
        sh:minCount 1 ;
    ] .
"""

In [ ]:
# save the shapes graph
with open("DEFINED/PATH", "w") as text_file:
    print(shapes_graph, file=text_file)

## Automated

In [ ]:
prefixes = """
@prefix : <http://example.org/ns#> .
@prefix dash: <http://datashapes.org/dash#> .
@prefix dbc: <http://dbpedia.org/resource/Category:> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbp: <http://dbpedia.org/property/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/ns#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix person: <http://dbpedia.org/ontology/Person> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix sock: <https://cs.ui.ac.id/ns/sock#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .
"""

In [ ]:
def construct_node_shape(node_shape_name, target, is_class_target=True):
  """
  Construct the node shape.

  Parameters
  ----------
  node_shape_name : str
    A name for a node shape
  target : str
    An IRI as a target of node shape 
  is_class_target : bool, optional
    A boolean value to determine the use of target class or target node (default is True)

  Returns
  -------
  str
    A node shape in a type of string
  """

  if is_class_target:
    return f"""
ex:{node_shape_name}
    a sh:NodeShape ;
    sh:targetClass {target} ;
"""
  return f"""
ex:{node_shape_name}
    a sh:NodeShape ;
    sh:targetNode {target} ;
"""

In [ ]:
def construct_property_shape(df, prop_col, card_col):
  """
  Construct the property shape.

  Parameters
  ----------
  df : DataFrame
    A name for a node shape
  prop_col : str
    A column name of a property 
  card_col : str
    A column name of a property's cardinality

  Returns
  -------
  str
    A property shape in a type of string
  """

  property_shape = ""

  for idx, row in df.iterrows():
    shape = f"""
sh:property [ a sh:PropertyShape;
        sh:path <{row[prop_col]}>;
        sh:minCount {row[card_col]} ];
"""
    property_shape += '    ' + shape[1:]

  # correct the last symbol
  property_shape = property_shape[:-2] + '.'

  return property_shape

In [ ]:
def construct_shapes_graph(node_shape_name, df, prop_col, card_col, target, is_class_target=True):
  """
  Construct the property shape.

  Parameters
  ----------
  node_shape_name : str
    A name for a node shape
  df : DataFrame
    A name for a node shape
  prop_col : str
    A column name of a property 
  card_col : str
    A column name of a property's cardinality
  target : str
    An IRI as a target of node shape 
  is_class_target : bool, optional
    A boolean value to determine the use of target class or target node (default is True)

  Returns
  -------
  str
    A shapes shape in a type of string
  """

  # create node shape
  node_shape = construct_node_shape(node_shape_name, target, is_class_target)

  # choose a column to be a property
  if prop_col is None:
    prop_col = df.columns.tolist()[0]

  # choose a column to be a cardinality of a property
  if card_col is None:
    card_col = df.columns.tolist()[1]

  # create property shape
  property_shape = construct_property_shape(df, prop_col, card_col)

  # merge property shape with node shape
  shapes_graph = prefixes[1:] + node_shape + '    ' +property_shape[1:]

  return shapes_graph

In [ ]:
# Example
print(construct_shapes_graph("HotelShape", df, "prop.value", "cardinality", "dbo:Hotel"))

@prefix : <http://example.org/ns#> .
@prefix dash: <http://datashapes.org/dash#> .
@prefix dbc: <http://dbpedia.org/resource/Category:> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbp: <http://dbpedia.org/property/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/ns#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix person: <http://dbpedia.org/ontology/Person> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix sock: <https://cs.ui.ac.id/ns/sock#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/

## Spreadsheet

In [ ]:
# load a spreadsheet file in a format of CSV
df = pd.read_csv("DEFINED/PATH")

## Ontology

In [ ]:
def get_property_by_ontology(class_name):
  """
  Get all the desired properties of a class by an ontological approach.

  Parameters
  ----------
  class_name : str
    A name of a class 

  Returns
  -------
  DataFrame
    A table consisting of the desired properties
  """

  query = f"""
SELECT DISTINCT ?prop
WHERE {{
  ?prop rdfs:domain {class_name} .
}}
"""

  prop_df = query_sparql(query, "http://dbpedia.org/sparql")
  prop_df['cardinality'] = 1

  return prop_df

In [ ]:
# example
df = get_property_by_ontology("dbo:Hotel")
df

,prop.type,prop.value,cardinality
0,uri,http://dbpedia.org/ontology/numberOfParkingSpaces,1
1,uri,http://dbpedia.org/ontology/numberOfRestaurants,1
2,uri,http://dbpedia.org/ontology/numberOfSuites,1
3,uri,http://dbpedia.org/ontology/starRating,1


## Statistics

In [5]:
def get_property_by_statistics(class_name):
  """
  Get all the desired properties of a class by an statistical approach.

  Parameters
  ----------
  class_name : str
    A name of a class 

  Returns
  -------
  DataFrame
    A table consisting of the desired properties
  """

  # get candidate properties
  query = f"""
SELECT DISTINCT ?p
WHERE {{
   ?s a {class_name} ;
        ?p [] .
    FILTER(isUri(?p) && STRSTARTS(STR(?p), STR(dbo:)))
}}
"""
  candidate_prop = query_sparql(query, "http://dbpedia.org/sparql")


  # get number of entities of a class
  query = f"""
SELECT (COUNT(DISTINCT ?entity) AS ?numOfEntities)
WHERE {{
  ?entity a {class_name} .
}}
"""
  num_of_entities = int(query_sparql(query, "http://dbpedia.org/sparql").iloc[0,2])


  # query the frequency of all the properties
  list_rel_freq = []

  for _, row in candidate_prop.iterrows():
    prop = row['p.value']

    query = f"""
SELECT (COUNT(DISTINCT ?entity) AS ?numOfEntities)
WHERE {{
  ?entity a {class_name} ;
          <{ prop }> [] .
}}
"""

    # count the relative frequency
    num_of_union = int(query_sparql(query, "http://dbpedia.org/sparql").iloc[0,2])
    list_rel_freq.append(num_of_union / num_of_entities)
  

  # arrange the result
  prop = candidate_prop.copy()
  prop['rel_freq'] = list_rel_freq
  prop['cardinality'] = 1
  prop.sort_values('rel_freq', ascending=False, inplace=True)
  prop.reset_index(drop=True, inplace=True)

  return prop.head(10)

In [6]:
# Example
df = get_property_by_statistics("dbo:University")
df

,p.type,p.value,rel_freq,cardinality
0,uri,http://dbpedia.org/ontology/wikiPageWikiLink,0.944675,1
1,uri,http://dbpedia.org/ontology/wikiPageID,0.944631,1
2,uri,http://dbpedia.org/ontology/wikiPageLength,0.944631,1
3,uri,http://dbpedia.org/ontology/wikiPageRevisionID,0.943485,1
4,uri,http://dbpedia.org/ontology/abstract,0.758560,1
5,uri,http://dbpedia.org/ontology/wikiPageExternalLink,0.650201,1
6,uri,http://dbpedia.org/ontology/city,0.505200,1
7,uri,http://dbpedia.org/ontology/thumbnail,0.504164,1
8,uri,http://dbpedia.org/ontology/type,0.377980,1
9,uri,http://dbpedia.org/ontology/country,0.346781,1


# Data Validation

In [ ]:
# load data containing all the property values for the entities
data_prop = pd.read_csv("DEFINED/PATH")

# handle NaN values in language attribute
# if there is no column "o.xml:lang", just skip it
data_prop[['o.xml:lang']] = data_prop[['o.xml:lang']].fillna('not specified')

# convert data into data graph
data_graph = Graph()

# add instance relation for all entities
# only used for checking with target for a certain class
# if not, just skip it
for _, row in data.iterrows():
  s = URIRef(row['entity.value'])
  p = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')
  o = URIRef('http://dbpedia.org/ontology/Country') # example: Country class
  data_graph.add((s, p, o))

# add node-property relation for all entities
for _, row in data_prop.iterrows():
  s = URIRef(row['s.value'])
  p = URIRef(row['p.value'])
  if row['o.type'] == 'literal':
    if 'o.xml:lang' not in data_prop.columns.tolist() or row['o.xml:lang'] == 'not specified':
      o = o = Literal(row['o.value'])
    else:
      o = Literal(row['o.value'], lang=row['o.xml:lang'])
  elif row['o.type'] == 'typed-literal':
    o = Literal(row['o.value'], datatype=row['o.datatype'])
  else:
    o = URIRef(row['o.value'])
  data_graph.add((s, p, o))

In [ ]:
def build_data_graph(data, data_prop):
    # handle NaN values in language attribute
    # if there is no column "o.xml:lang", just skip it
    if 'o.xml:lang' in data_prop.columns.tolist():
        data_prop[['o.xml:lang']] = data_prop[['o.xml:lang']].fillna('not specified')

    # convert data into data graph
    data_graph = Graph()

    # add instance relation for all entities
    # only used for checking with target for a certain class
    # if not, just skip it
    for _, row in data.iterrows():
        s = URIRef(row['entity.value'])
        p = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')
        o = URIRef('http://dbpedia.org/ontology/Country') # example: Country class
        data_graph.add((s, p, o))

    # add node-property relation for all entities
    for _, row in data_prop.iterrows():
        s = URIRef(row['s.value'])
        p = URIRef(row['p.value'])
        if row['o.type'] == 'literal':
            if 'o.xml:lang' not in data_prop.columns.tolist() or row['o.xml:lang'] == 'not specified':
                o = o = Literal(row['o.value'])
            else:
                o = Literal(row['o.value'], lang=row['o.xml:lang'])
        elif row['o.type'] == 'typed-literal':
                o = Literal(row['o.value'], datatype=row['o.datatype'])
        else:
                o = URIRef(row['o.value'])
        data_graph.add((s, p, o))

    return data_graph

In [ ]:
# load shapes graph
def build_shapes_graph(path):
    return Graph().parse(path)

In [ ]:
def validate_graph(shapes_graph, data_graph, is_advanced=False):
  """
  Validate the data graph over the shapes graph with the SHACL engine provided by PySHACL.
  
  Parameters
  ----------
  shapes_graph : Graph
    The shapes graph containing all the constraints
  data_graph : Graph
    The data graph containing all the instances to be validated along with their property values
  is_advanced : boolean, optional
    ...

  Returns
  -------
  (bool, Graph, str)
    value of conformation, validation report in the shape of a graph, and
    validation report in the shape of a text
  """

  result = validate(
    data_graph = data_graph,
    shacl_graph = shapes_graph,
    advanced = is_advanced,
    serialize_report_graph="ttl",
    )
  
  return result

# validate the data graph
conforms, report_graph, report_text = validate_graph(shapes_graph, data_graph)

In [ ]:
def create_report_validation(df, use_col, report_graph, prop_list):
  """
  Transform validation result from a form of graph to a form of table (Pandas dataframe).

  Parameters
  ----------
  df : DataFrame
    A table containing all the instances to be validated
  use_col : list
    A sublist of the variable list_prop used for lookup through the report graph 
  report_graph : Graph
    A validation report in the shape of graph as a result of SHACL validation
  prop_list : list
    A list of property to be checked

  Returns
  -------
  DataFrame
    A table consisting of instances with the value of validation
  """

  report = Graph()
  report.parse(data=report_graph)

  # list all incompleteness
  list_incomplete = []

  for prop in prop_list:
    report_query = f"""
PREFIX dbo: <http://dbpedia.org/ontology/>
SELECT ?focusNode
WHERE {{
[] <http://www.w3.org/ns/shacl#result> ?id .
?id <http://www.w3.org/ns/shacl#focusNode> ?focusNode ;
    <http://www.w3.org/ns/shacl#resultPath> {prop} .
}}
"""
    res = report.query(report_query)

    list_entities = []
    for row in res:
      list_entities.append([str(row.focusNode), 0])

    list_incomplete.append(list_entities)

  # convert to dict of incompleteness
  validation = df[[use_col]]
  incomplete_dict = dict()

  for idx, prop in enumerate(prop_list):
    incomplete_dict[f"df_incomplete_{prop}"] = pd.DataFrame(list_incomplete[idx], columns=[use_col, prop])

  # merge the information
  for key in incomplete_dict.keys():
    validation = pd.merge(validation, incomplete_dict[key], on=use_col, how='left').fillna(1)

  validation['complete_all'] = validation.iloc[:,1:].sum(axis=1)/len(prop_list)

  return validation

prop_list = ['rdfs:label', 'rdfs:comment', 'dbo:abstract']
validation = create_report_validation(data, "entity.value", report_graph, prop_list)
validation.to_csv(path+"DBP-LDC01M-20220518-Country-validation.csv", index=False)

# Visualization

In [ ]:
def create_completeness_info_viz(validation, prop_list, title):
  """
  Visualize the result validation

  Parameters
  ----------
  validation : DataFrame
    A table consisting of validated instances
  prop_list : list
    A list of properties to be checked
  title : str
    A title to be shown in the chart
  """

  # transform validation data into suitable one for the chart
  res_list = []
  for prop in prop_list:
    res = validation[prop].value_counts(normalize=True).rename_axis('label').reset_index(name='percentage')
    res['property'] = prop
    res_list.append(res)

  comp_summary = pd.concat(res_list, ignore_index=True, sort=False)

  # transform value of 1 and 0
  dict_map = {1: 'Complete', 0: 'Incomplete'}
  comp_summary['label'] = comp_summary['label'].map(dict_map)

  # create stacked bar plot
  fig = px.bar(comp_summary,
             x='percentage',
             y='property',
             color='label',
             title=title)

  # adjust plot layout
  fig.update_layout(
      autosize=False,
      width=800,
      height=100*len(prop_list) if len(prop_list) > 3 else 300,
      xaxis = dict(
          tickmode = 'array',
          tickvals = [0, 0.2, 0.4, 0.6, 0.8, 1],
          ticktext = ['0%', '20%', '40%', '60%', '80%', '100%']
      ))

  fig.show()

# list all the property to be visualized
prop_list = ['...']
# prop_list = ['rdfs:label', 'rdfs:comment', 'dbo:abstract'] # example


create_completeness_info_viz(validation,
                             prop_list,
                             '...',) # insert title